In [24]:
import pandas as pd
import numpy as np

interesting keywords for Google trends:
- debt
- stocks
- inflation
- portfolio
- housing

- revenues
- employment
- unemployment
- leverage
- credit

- return
- bonds
- economics
- religion
- money

- profit
- investment
- gains
- cancer
- marriage

- buy
- hold
- headlines
- present
- crisis

- society
- gross
- loss
- hedge
- nasdaq

- cash
- short sell
- greed
- gain
- happy

- financial markets
- finance
- house
- water
- rich

- risk
- oil
- opportunity
- golf
- invest

- success
- economy
- food
- war
- consume

- nyse
- movie
- health
- consumption
- transaction

- environment
- fond

Adding companies that belong to S&P500 (https://fknol.com/list/market-cap-sp-500-index-companies.php)

- apple
- microsoft
- amazon

- alphabet
- google
- facebook
- berkshire
- visa

- walmart
- johnson
- j p morgan
- procter and gamble
- mastercard

- nvidia
- united health
- home depot
- walt disney
- verizon communications

- paypal
- bank of america
- comcast
- adobe
- coca cola

- pfizer
- netflix
- nike
- intel
- at&t

- merck
- salesforce
- pepsico
- abbott labs
- cisco

- oracle
- mcdonalds
- medtronic


In [63]:
df0 = pd.read_csv('data/data.csv', header=[1])
df0.set_index('Mois')

,debt: (États-Unis),stocks: (États-Unis),inflation: (États-Unis),portfolio: (États-Unis),housing: (États-Unis)
Mois,,,,,
2004-01,29,38,10,16,73
2004-02,29,31,10,14,74
2004-03,28,28,10,15,80
2004-04,27,28,11,15,83
2004-05,28,23,12,16,80
...,...,...,...,...,...
2020-08,25,50,7,11,64
2020-09,27,43,9,11,54
2020-10,27,38,9,10,55


In [66]:
import glob

all_files = glob.glob("data/*")

dfs = []

for filename in all_files:
    print(filename)
    df = pd.read_csv(filename, header=[1])
    df = df.set_index('Mois')
    dfs.append(df)



data/data18.csv
data/data3.csv
data/data2.csv
data/data1.csv
data/data5.csv
data/data4.csv
data/data6.csv
data/data7.csv
data/data.csv
data/data9.csv
data/data8.csv
data/data14.csv
data/data15.csv
data/data17.csv
data/data16.csv
data/data12.csv
data/data13.csv
data/data11.csv
data/data10.csv


In [73]:
df = pd.concat(dfs, axis=1)

In [77]:
cols = [el.split(':')[0] for el in list(df.columns)]
df.columns = cols

In [78]:
df.head()

,oracle,mcdonalds,medtronic,tourism,bank,profit,investment,gains,cancer,marriage,...,environment,fond,apple,microsoft,amazon,nyse,movie,health,consumption,transaction
Mois,,,,,,,,,,,,,,,,,,,,,
2004-01,8,2,< 1,6,50,11,30,4,79,40,...,2,< 1,10,24,10,1,68,37,1,1
2004-02,8,2,< 1,6,48,12,30,3,81,57,...,3,< 1,10,24,9,1,61,37,1,1
2004-03,8,3,< 1,5,51,11,30,4,82,56,...,2,< 1,10,21,9,1,61,35,1,1
2004-04,8,3,< 1,6,50,12,28,4,83,47,...,3,< 1,9,20,9,1,58,35,1,1
2004-05,8,3,< 1,6,51,10,23,3,84,45,...,2,< 1,9,22,9,1,63,34,2,1
